#### Importing libraries

In [45]:
import xlwings as xw
from datetime import datetime
import openpyxl
from openpyxl.utils import get_column_letter
import numpy as np
import re
import pandas as pd


##### Opening files

In [46]:
#Reading files
user = input("Enter your laptop user: ")
# Get the current date
current_date = datetime.now().strftime("%Y%m%d")

gl = xw.Book(r'C:\Users\{user}\Downloads\general_ledger-{çurrent_day}.xlsx'.format(user=user, çurrent_day=current_date))
sec_dep_det = xw.Book(r'C:\Users\{user}\Downloads\security_deposit_funds_detail-{çurrent_day}.xlsx'.format(user=user,çurrent_day=current_date))

# Indicating worksheet
gl_worksheet = gl.sheets['Sheet1']
sec_dep_det_worksheet = sec_dep_det.sheets['Sheet1']

#### Diagnostics 1

##### Including adjusted payee names

In [47]:
# Defining rows to calculate values
first_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 2 # first row of values
last_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row - 1  # last row of values



# Creating columns to adjust payee names
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting column
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for payee names


# Including adjusted payee names

# Read values from column C and remove spaces using pandas
data_range_for_payee = gl.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values, last_row_of_values))
column_c_values = data_range_for_payee.value

# Convert to a pandas DataFrame for efficient processing
df = pd.DataFrame(column_c_values, columns=['Column_C'])

# Remove spaces and handle empty values
df['Column_D'] = df['Column_C'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else "No Payee Name")

# Write the modified data back to column D & E (including numerical index)
gl.sheets['Sheet1'].range('D{}'.format(first_row_of_values)).value = dict(df['Column_D'])
gl.sheets['Sheet1'].range('E{}'.format(first_row_of_values - 2)).value = 'Payee'





##### Adding net column (debt - credit)

In [48]:
# Adding net column (debt - credit)
gl.sheets['Sheet1'].api.Columns(10).Insert() # Inserting column

# Set the formula for the entire range of cells in column J
range_formula = '=H{}:H{}-I{}:I{}'.format(first_row_of_values, last_row_of_values, first_row_of_values, last_row_of_values)
# Applying the formula to the entire column from the starting row to the ending row
gl.sheets['Sheet1'].range('J{}:J{}'.format(first_row_of_values, last_row_of_values)).formula = range_formula 
gl.sheets['Sheet1'].range('J{}'.format(first_row_of_values - 2)).value = 'Net'


##### Funds detail tenant name

In [49]:
sec_dep_det.sheets['Sheet1'].api.Columns(3).Insert() # Inserting column
sec_dep_det.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for tenant names



# Defining rows to calculate values
first_row_of_values_sec_dep = sec_dep_det.sheets['Sheet1'].range('a'+ str(sec_dep_det.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 1 # first row of values
last_row_of_values_sec_dep = sec_dep_det.sheets['Sheet1'].range('a'+ str(sec_dep_det.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row  # last row of values


# Read the data into a pandas DataFrame
column_data = sec_dep_det.sheets['Sheet1'].range('b{}:b{}'.format(first_row_of_values_sec_dep, last_row_of_values_sec_dep)).value
df = pd.DataFrame(column_data, columns=['Names'])

# Rearrange the names efficiently using pandas
df['Names'] = df['Names'].str.split(', ').str[::-1].str.join(' ')

# Write the updated data back to Excel
sec_dep_det.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values_sec_dep, last_row_of_values_sec_dep)).value = dict(df['Names'])


#### Creating pivot table

In [50]:
# Open a new Excel workbook
diagnostics_1_wb = xw.Book()

In [51]:
# Property Name
diagnostics_1_wb.sheets['Sheet1'].range('a1:e1').api.Font.Bold = True
gl.sheets['Sheet1'].range('m{}:m{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('A2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('A1').value = 'Property Name'

# Payee

gl.sheets['Sheet1'].range('e{}:e{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('b2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('b1').value = 'Property Name'

# Payee
gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).formula = "=SUMIFS($J${first_row}:$J${last_row},$E${first_row}:$E${last_row},E{first_row},$M${first_row}:$M${last_row},M{first_row})".format(first_row=first_row_of_values, last_row= last_row_of_values)
values_list = gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).value
result_dict = {index: value for index, value in enumerate(values_list)}
diagnostics_1_wb.sheets['Sheet1'].range('c2').value = result_dict
diagnostics_1_wb.sheets['Sheet1'].range('c:c').api.Delete() # Deleting column C
diagnostics_1_wb.sheets['Sheet1'].range('c1').value = 'Sum of Net'




#### Notes

- No cambiarle el nombre a los archivos que se descargan de appfolio

- no tener en la carpeta de descargas archivos previos a este proceso con los siguientes nombres
general_ledger-20231110

    - security_deposit_funds_detail-20231110

    - delinquency-20231110